#### Autograd：自动求导机制
PyTorch 中所有神经网络的核心是 autograd 包。 我们先简单介绍一下这个包，然后训练第一个简单的神经网络。

autograd包为张量上的所有操作提供了自动求导。 它是一个在运行时定义的框架，这意味着反向传播是根据你的代码来确定如何运行，并且每次迭代可以是不同的。

#### 张量

torch.Tensor是这个包的核心类，如果设置`.requires_grad`为True，那么将跟踪所有对于该张量的操作。当完成计算后通过调用`.backward()`，自动计算所有的梯度，这个张量的所有梯度将会自动累加到`.grad`属性

在自动梯度计算中还有另一个重要的类Function。
Tensor与Function互相连接并生成一个非循环图，它表示和存储了完整的计算历史。每个张量都有一个`.grad_fn`属性，这个属性引用了一个创建了Tensor的Function（除非这个张量是用户手动创建的，这时这个张量的`grad_fn`为None

如果需要计算导数，可以在Tensor上调用`.backward()`。如果Tensor是一个标量（即它包含一个元素数据）则不需要为`backward()`指定任何参数，但是如果它有更多的元素，==需要指定一个`gradient`参数来匹配张量的形状==。

In [1]:
import torch

创建一个张量，并追踪它的计算历史

In [2]:
x = torch.ones(2, 2, requires_grad = True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


对张量进行操作

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


结果y已经被计算出来了，所以，`grad_fn`已经被自动生成了

In [5]:
print(y.grad_fn)

In [6]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`.requires_grad()`可以改变现有张量的`requires_grad`属性。如果没有指定的话，默认输入的flag是False

In [8]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


#### 梯度
反向传播，因为out是一个标量(scalar)，out.backward()等价于out.backward(torch.tensor(1))

In [9]:
out.backward()
# print gradients d(out)/dx
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


==下面不太懂==
$out = \frac{1}{4}\sum_{i}z_i, z_i = 3(x_i+2)^2 and z_i|_{x_i=1}=27$
因此,$\frac{\partial o}{\partial x_i}=\frac{3}{2}(x_i+2),hence\frac{\partial o}{\partial x_i}|_{x_i=1}=\frac{9}{2}=4.5$

可以用autograd做更多的操作

In [15]:
x = torch.randn(3, requires_grad = True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([-534.7850,  -32.8364, 1520.0532], grad_fn=<MulBackward0>)


In [16]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype = torch.float)
y.backward(gradients)
print(x.grad)


tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


如果，`.requires_grad=True`而你又不希望进行autograd的计算，那么可以将变量包裹在`with torch.no_grad()`中

In [17]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


疑难解答
----
----
1.requires_grad

若一个节点的被设置为True，则所有依赖它的节点的都为True

----
2.`.backward()`
以下参考自：[PyTorch中的自动求导backward()所需参数含义](https://www.cnblogs.com/JeasonIsCoding/p/10164948.html)
如果out.backward()中的out是一个标量的话（相当于神经网络有一个样本，这个样本有两个属性，神经网络有一个输出），那么此时我的backward函数是不需要任何输入参数的。
如果out是一个向量（或者理解为1*N的矩阵）的话，我们传入的参数，是对原来模型正常求导出来的雅可比矩阵进行线性操作，可以把我们传进的参数看成是一个行向量，那么我们得到的结果为：
$$
(arg \times Jacobian)
$$
![](https://raw.githubusercontent.com/CallMeNumber1/FigureBed/windows/img/20190917203500.png)

总结，因为经过了复杂的神经网络之后，out中每个数值都是由很多输入样本的属性线性或非线性组合而成的，那么out中的每个数值和输入数据中的每个数值都有关联，也就是说out中的每个数可以对a中每个数求导（即得到雅可比矩阵），那么我们backward()的参数[k1,k2,...,kn]的含义就是：
![](https://raw.githubusercontent.com/CallMeNumber1/FigureBed/windows/img/20190917203550.png)
也可以理解为每个out分量对$a_n$求导时的权重（对$a_i$的权重即为$k_i$）

也可以扩展一下多个样本的分类问题，k的维度应该是`输入样本的个数*分类的个数`

torch.mm(a, b)    # 矩阵a和矩阵b相乘

torch.mul(a, b)   # 矩阵a和矩阵b中对应元素相乘（即要求a和b维度相同）